In [1]:
import os
import PIL
import dlib
# import matplotlib.pyplot as plt
import datetime
import numpy as np

from sklearn.metrics import confusion_matrix
# import seaborn as sns
import pandas as pd

In [2]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split, SubsetRandomSampler
import torch.optim as optim

import tensorboardX

In [3]:
torch.manual_seed(0)
np.random.seed(0)

In [4]:
epochs = 10
batch_size = 16
lr = 1e-4

In [5]:
logname = f'resnet_{batch_size}_{epochs}_{lr}_{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}'

if not os.path.exists(os.path.join('models', logname)):
    os.makedirs(os.path.join('models', logname))

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [8]:
data_root = 'images/dataset'
save_path = 'images/cropped_faces'

In [9]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
class FaceDataset(Dataset):
    def __init__(self, n_classes=4, root_dir=save_path, transform=transform):
        self.root_dir = root_dir
        self.transform = transform
        self.files = os.listdir(self.root_dir)
        if n_classes == 4:
            self.labels = [int(file.split('_')[0]) for file in self.files]
        
        self.n_classes = n_classes
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.files[idx])
        img = PIL.Image.open(img_name)
        img = self.transform(img)
        label = int(self.files[idx].split('_')[0])
        label = label if self.n_classes == 4 else label // 2
        return img, label

In [11]:
dataset = FaceDataset(n_classes=4)
train_data = dataset

# len_data = len(dataset)
# train_size = int(0.8 * len_data)
# test_size = len_data - train_size

# train_data, test_data = random_split(dataset, [train_size, test_size])
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# print('Train:', len(train_data), 'Test:', len(test_data))

In [12]:
writer = tensorboardX.SummaryWriter(logdir=f'runs/{logname}'+ datetime.datetime.now().strftime('%Y%m%d%H%M%S'))

In [13]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        writer.add_scalar('Loss/train', loss.item(), epoch * len(train_loader) + i)
        writer.add_scalar('Accuracy/train', 100 * correct / total, epoch * len(train_loader) + i)
        
        if i % 100 == 99:
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100}')
            running_loss = 0.0
            correct = 0
            total = 0
            
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    print(f'Train {epoch+1}/{epochs} Loss: {train_loss} Accuracy: {train_accuracy}')
    
    return train_loss, train_accuracy

In [14]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
model.fc = nn.Linear(512, 4)
model = model.to(device)

# # transfer learning
# for param in model.parameters():
#     param.requires_grad = True
# in_features = model.fc.in_features
# model.fc = nn.Linear(in_features, 4)

Using cache found in C:\Users\pomat/.cache\torch\hub\pytorch_vision_v0.6.0
c:\Users\pomat\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pomat\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [16]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        writer.add_scalar('train_loss', loss.item(), i*epoch)
        writer.add_scalar('train_accuracy', 100*correct/total, i*epoch)
        
        if i % 10 == 99:
            print(f'[{i+1}, {running_loss/100:.3f}]')
            running_loss = 0.0
            correct = 0
            total = 0
        
    train_loss = running_loss/len(train_loader)
    train_accuracy = 100*correct/total
    print(f'Training {epoch+1}/{epochs}:\tTrain loss: {train_loss:.3f},\tTrain acc: {train_accuracy:.3f}')
    
    return train_loss, train_accuracy

In [17]:
y_pred = []
y_true = []
    
def test(model, test_loader, criterion):
    model.eval()
    
    total_loss = 0
    
    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            y_pred.extend(predicted.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
            
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
    test_loss = total_loss / len(test_loader)
    test_acc = 100 * sum([1 for i in range(len(y_pred)) if y_pred[i] == y_true[i]]) / len(y_true)
    
    writer.add_scalar('test_accuracy', test_acc)
    writer.add_scalar('test_loss', test_loss)
    
    return test_loss, test_acc

In [18]:
# def train_val_curve(epoch, fold, train_losses, train_accuracies, val_losses, val_accuracies):
#     fig, ax = plt.subplots(2, 1, figsize=(10, 10))
    
#     ax[0].plot(range(epochs*fold), train_losses, label='train')
#     ax[0].plot(range(epochs*fold), val_losses, label='val')
#     ax[0].set_title('Loss')
#     ax[0].legend()
    
#     ax[1].plot(range(epochs*fold), train_accuracies, label='train')
#     ax[1].plot(range(epochs*fold), val_accuracies, label='val')
#     ax[1].set_title('Accuracy')
#     ax[1].legend()
    
#     plt.show()
    
#     writer.add_figure('train_val_curve', fig, epoch*fold)

In [19]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=0)

In [20]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight.data)

In [21]:
train_losses_folds, train_accs_folds = [], []
test_losses_folds, test_accs_folds = [], []

In [22]:
def run():
    for fold, (train_idx, test_idx) in enumerate(kfold.split(train_data)):
        print(f'Fold {fold+1}/{5}')
        
        train_subset = SubsetRandomSampler(train_idx)
        test_subset = SubsetRandomSampler(test_idx)
        
        train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_subset)
        test_loader = DataLoader(train_data, batch_size=batch_size, sampler=test_subset)
        
        model.apply(weights_init)
        
        optimizer = optim.Adam(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()
        
        train_losses = []
        train_accuracies = []
        
        for epoch in range(epochs):
            
            train_loss, train_acc = train(model, train_loader, optimizer, criterion, epoch)
            
            train_losses.append(train_loss)
            train_accuracies.append(train_acc)
            
        train_losses_folds.append(sum(train_losses) / len(train_losses))
        train_accs_folds.append(sum(train_accuracies) / len(train_accuracies))
        print(f'Train {fold+1}/{5}:\tTrain loss: {train_losses_folds[-1]:.3f},\tTrain acc: {train_accs_folds[-1]:.3f}')
        
        test_loss, test_acc = test(model, test_loader, criterion)
        print(f'Test {fold+1}/{5}:\tTest loss: {test_loss:.3f},\tTest acc: {test_acc:.3f}')
        
        test_losses_folds.append(test_loss)
        test_accs_folds.append(test_acc)
                
    final_train_loss = sum(train_losses_folds) / len(train_losses_folds)
    final_train_acc = sum(train_accs_folds) / len(train_accs_folds)
    
    final_test_loss = sum(test_losses_folds) / len(test_losses_folds)
    final_test_acc = sum(test_accs_folds) / len(test_accs_folds)
    
    return final_train_loss, final_train_acc, final_test_loss, final_test_acc

In [23]:
# plt.figure(figsize=(10, 5))
# plt.plot(train_accuracies, label='Train Accuracy')
# plt.plot(val_accuracies, label='Val Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.title('Train and Val Accuracy')
# plt.legend()
# plt.show()

# plt.figure(figsize=(10, 5))
# plt.plot(train_losses, label='Train Loss')
# plt.plot(val_losses, label='Val Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Train and Val Loss')
# plt.legend()
# plt.show()

In [24]:
train_loss, train_acc, train_loss, train_acc = run()

Fold 1/5


C:\Users\pomat\AppData\Local\Temp\ipykernel_66656\1855755017.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight.data)


Training 1/10:	Train loss: 1.214,	Train acc: 45.191
Training 2/10:	Train loss: 1.034,	Train acc: 55.741
Training 3/10:	Train loss: 0.965,	Train acc: 59.053
Training 4/10:	Train loss: 0.903,	Train acc: 61.310
Training 5/10:	Train loss: 0.807,	Train acc: 66.585
Training 6/10:	Train loss: 0.709,	Train acc: 70.486
Training 7/10:	Train loss: 0.563,	Train acc: 77.380
Training 8/10:	Train loss: 0.431,	Train acc: 83.734
Training 9/10:	Train loss: 0.292,	Train acc: 88.984
Training 10/10:	Train loss: 0.196,	Train acc: 93.302
Train 1/5:	Train loss: 0.711,	Train acc: 70.177
Test:	Test loss: 0.988,	Test acc: 71.078
Test 1/5:	Test loss: 0.988,	Test acc: 71.078
Fold 2/5
Training 1/10:	Train loss: 1.232,	Train acc: 45.303
Training 2/10:	Train loss: 1.060,	Train acc: 55.237
Training 3/10:	Train loss: 0.965,	Train acc: 59.284
Training 4/10:	Train loss: 0.896,	Train acc: 61.099
Training 5/10:	Train loss: 0.829,	Train acc: 64.876
Training 6/10:	Train loss: 0.749,	Train acc: 68.972
Training 7/10:	Train los

In [25]:
classes = ['Spring', 'Summer', 'Fall', 'Winter'] if dataset.n_classes == 4 else ['Warm', 'Cool']

In [26]:
cm = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cm, index=classes, columns=classes)

In [27]:
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('True')
writer.add_figure('confusion_matrix', plt.gcf())

NameError: name 'plt' is not defined